# Backpropagation
This notebook will show you the underlying implementation of Backpropagation for a simple neural network with 3 layers(input_layer, hidden_layer, output_layer).

If you have any suggestions, welcome to contact [hushenglang@gmail.com](), or pull request on github.

### Data Preparation

In [20]:
# data_prep module is data_prep.py
from data_prep import *
features, targets, features_test, targets_test = prepare_data()

In [21]:
features.head()

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0


In [22]:
targets.head()

209    0
280    0
33     1
210    0
93     0
Name: admit, dtype: int64

### Backpropagation Implementation

loss function used in this implementaion is **MSE**

for sure, you can change to use other loss function like **cross-entropy**

<img src="asset/back_propagation1.png",width=500,height=500, style="float: left;">

<img src="asset/back_propagation2.png",width=200,height=200, style="float: left;">

<img src="asset/back_propagation3.png",width=200,height=200, style="float: left;">

**chain rule** to backpropagate to calculate **w**

<img src="asset/back_propagation4.png",width=600,height=600, style="float: left;">

Backpropagation is a leaky abstraction.

It may cause Vanishing gradients on sigmoids, Dying ReLUs and Exploding gradients in RNNs.

please refer to this post for details:

https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b

In [23]:
# Defining the sigmoid function for activations
def sigmoid(x):
    return 1/(1+np.exp(-x))

# backpropagation with batch gradient descent
def backprop_batch_gradient_descent(weights_input_hidden, weights_hidden_output, features, targets):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_activations = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_activations, weights_hidden_output))

        ## Backward pass ##
        # Calculate error gradient in output unit
        output_error_gradient = (y-output) * output * (1-output)

        # propagate errors to hidden layer hidden
        hidden_error_gradient = weights_hidden_output * output_error_gradient * hidden_activations * (1-hidden_activations)

        # Update the change in weights
        del_w_hidden_output +=  output_error_gradient * hidden_activations
        del_w_input_hidden +=  hidden_error_gradient * x[:, None]

    # Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

### Training Network

In [24]:
# Hyperparameters
n_hidden = 20  # number of hidden units
epochs = 1000
learnrate = 0.05

n_records, n_features = features.shape
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [25]:
def train():
    for e in range(epochs):
        backprop_batch_gradient_descent(weights_input_hidden, weights_hidden_output, features, targets)
        # Printing out the mean square error on the training set
        if e % (epochs / 10) == 0:
            hidden_activations = sigmoid(np.dot(features, weights_input_hidden))
            out = sigmoid(np.dot(hidden_activations,
                                 weights_hidden_output))
            loss = np.mean((out - targets) ** 2)         
            print("Train loss: ", loss)

train()

Train loss:  0.228492837836
Train loss:  0.220652879861
Train loss:  0.217154284272
Train loss:  0.214766019789
Train loss:  0.212850794833
Train loss:  0.21126659987
Train loss:  0.209941781907
Train loss:  0.208823298017
Train loss:  0.207869565384
Train loss:  0.207047917328


In [26]:
# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725


### Further Reading
Understand backprop:

https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b

https://www.youtube.com/watch?v=59Hbtz7XgjM